In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.losses import BinaryFocalCrossentropy
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
base_path = "/content/drive/MyDrive/new_augmented_dataset"
print("Folder contents:", os.listdir(base_path))

Folder contents: ['real', 'fake']


In [4]:
FINAL_DATASET_DIR = '/content/drive/MyDrive/new_augmented_dataset'


real_paths = [p for p in glob(os.path.join(FINAL_DATASET_DIR, 'real', '*')) if os.path.exists(p)]
fake_paths = [p for p in glob(os.path.join(FINAL_DATASET_DIR, 'fake', '*')) if os.path.exists(p)]

print("Number of real images:", len(real_paths))
print("Number of fake images:", len(fake_paths))

all_paths = real_paths + fake_paths
all_labels = [0] * len(real_paths) + [1] * len(fake_paths)

Number of real images: 2368
Number of fake images: 2422


In [5]:
img_train_val, img_test, y_train_val, y_test = train_test_split(
    all_paths, all_labels, test_size=0.10, random_state=42, stratify=all_labels
)

img_train, img_val, y_train, y_val = train_test_split(
    img_train_val, y_train_val, test_size=0.2222, random_state=42, stratify=y_train_val
)

print("Training images:", len(img_train))
print("Validation images:", len(img_val))
print("Test images:", len(img_test))

Training images: 3353
Validation images: 958
Test images: 479


In [6]:
# ─── Settings ────────────────────────────────
TARGET_SIZE = (256, 256)
BATCH_SIZE  = 32

# ─── Preprocessing fn ─────────────────────────
def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, TARGET_SIZE)
    img = img / 255.0
    return img, label

data_augment = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.05),
  layers.RandomZoom(0.1),
  layers.RandomContrast(0.2),
])

# ─── 4d.  Dataset builder ──────────────────────────
def make_dataset(paths, labels, shuffle=False, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)

    # 1) Decode, resize, normalize
    ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    # 2) Cache the preprocessed images (so we don't re‑decode every epoch)
    ds = ds.cache()

    # 3) On‑the‑fly augmentation (only for training)
    if augment:
        ds = ds.map(
            lambda x, y: (data_augment(x, training=True), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )

    # 4) Batch and prefetch for performance
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds

# ─── Build splits ──────────────────────────────
train_ds = make_dataset(img_train, y_train, shuffle=True,  augment=True)
val_ds   = make_dataset(img_val,   y_val,   shuffle=False, augment=False)
test_ds  = make_dataset(img_test,  y_test,  shuffle=False, augment=False)

# ─── Quick sanity check ────────────────────────
print("Train batches:", tf.data.experimental.cardinality(train_ds).numpy(),
      "Val batches:",   tf.data.experimental.cardinality(val_ds).numpy())

Train batches: 105 Val batches: 30


In [7]:
# ───  DEFINE THE CNN ──────────────────────────────────
def build_currency_cnn(input_shape=(256, 256, 3), num_classes=1):
    reg = regularizers.l2(5e-4)
    inp = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=reg)(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=reg)(x)
    x = layers.MaxPooling2D()(x)

    # Block 2
    x = layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=reg)(x)
    x = layers.MaxPooling2D()(x)

    # Block 3
    x = layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu', kernel_regularizer=reg)(x)
    x = layers.GlobalAveragePooling2D()(x)

    # Dense head
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    return models.Model(inp, outputs, name='CurrencyCNN_NoAug')

cnn = build_currency_cnn()

for layer in cnn.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

cnn.compile(
  optimizer=tf.keras.optimizers.Adam(3e-4),
  loss=BinaryFocalCrossentropy(gamma=2.0),
  metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)
cnn.summary()

Model: "CurrencyCNN_NoAug"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 288,033 (1.10 MB)

 Trainable params: 287,137 (1.10 MB)

 Non-trainable params: 896 (3.50 KB)

In [8]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# ─── 3.  TRAIN FOR 25 EPOCHS ─────────────────────────────
history = cnn.fit(
    train_ds,
    validation_data = val_ds,
    epochs          = 25,
    verbose         = 1
)

Epoch 1/25
105/105 ━━━━━━━━━━━━━━━━━━━━ 562s 5s/step - accuracy: 0.5026 - auc: 0.4939 - loss: 0.3120 - val_accuracy: 0.5052 - val_auc: 0.5439 - val_loss: 0.2314
Epoch 2/25
105/105 ━━━━━━━━━━━━━━━━━━━━ 54s 516ms/step - accuracy: 0.5266 - auc: 0.5610 - loss: 0.2190 - val_accuracy: 0.5230 - val_auc: 0.6585 - val_loss: 0.2019
Epoch 3/25
105/105 ━━━━━━━━━━━━━━━━━━━━ 54s 519ms/step - accuracy: 0.6037 - auc: 0.6611 - loss: 0.1844 - val_accuracy: 0.7724 - val_auc: 0.8305 - val_loss: 0.1471
Epoch 4/25
105/105 ━━━━━━━━━━━━━━━━━━━━ 54s 517ms/step - accuracy: 0.7396 - auc: 0.7996 - loss: 0.1483 - val_accuracy: 0.6795 - val_auc: 0.8619 - val_loss: 0.1609
Epoch 5/25
105/105 ━━━━━━━━━━━━━━━━━━━━ 83s 522ms/step - accuracy: 0.7649 - auc: 0.8355 - loss: 0.1357 - val_accuracy: 0.7443 - val_auc: 0.8851 - val_loss: 0.1377
Epoch 6/25
105/105 ━━━━━━━━━━━━━━━━━━━━ 82s 520ms/step - accuracy: 0.7761 - auc: 0.8570 - loss: 0.1274 - val_accuracy: 0.7881 - val_auc: 0.8195 - val_loss: 0.1455
Epoch 7/25
105/105 ━━━━━

In [10]:
# ─── FINAL EVALUATION ────────
test_loss, test_acc, test_auc = cnn.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.3%}   |   Test AUC: {test_auc:.3%}   |   Test loss: {test_loss:.4f}")

Test accuracy: 91.232%   |   Test AUC: 91.149%   |   Test loss: 0.0871


In [12]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

# ── 1. Gather all true labels and model probabilities ──
y_true, y_prob = [], []
for imgs, labs in test_ds:
    probs = cnn.predict(imgs).flatten()    # or model.predict
    y_true.extend(labs.numpy().astype(int))
    y_prob.extend(probs.tolist())

y_true = np.array(y_true)
y_prob = np.array(y_prob)

# ── 2. Sweep thresholds and record metrics ──
thresholds = np.linspace(0.0, 1.0, 101)
results   = []

for t in thresholds:
    y_pred = (y_prob >= t).astype(int)
    f1  = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec  = recall_score(y_true, y_pred)
    results.append((t, f1, acc, prec, rec))

# ── 3. Pick the best threshold by F1 ──
best = max(results, key=lambda x: x[1])
best_t, best_f1, best_acc, best_prec, best_rec = best

print(f"Best threshold = {best_t:.3f}")
print(f"F1 = {best_f1:.3f}, Acc = {best_acc:.3f},  Precision = {best_prec:.3f}, Recall = {best_rec:.3f}")

# ── 4. Final classification report at best threshold ──
from sklearn.metrics import classification_report, confusion_matrix

y_pred_best = (y_prob >= best_t).astype(int)
print("\nClassification report @ best threshold:")
print(classification_report(y_true, y_pred_best, target_names=['Real','Fake']))

print("Confusion matrix:")
print(confusion_matrix(y_true, y_pred_best))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Best threshold = 0.530
F1 = 0.922, Acc = 0.914,  Precision = 0.855, Recall = 1.000

Classification report @ best threshold:
              precision    recall  f1-score   support

        Real       1.00      0.83      0.91       237
        Fake       0.86      1.00      0.92       242

    accuracy                           0.91       479
   macro avg       0.93      0.91      0.91       479
weighted avg       0.93

In [13]:
# Save full Keras model
cnn.save('/content/drive/MyDrive/new_cnn.keras')

# Save metadata (threshold, date, test metrics)
import json
meta = {
    "threshold": best_t,
    "test_accuracy": float(test_acc),
    "test_auc":      float(test_auc),
    "test_loss":     float(test_loss)
}
with open('/content/drive/MyDrive/new_cnn_meta.json','w') as f:
    json.dump(meta, f, indent=2)
